## **A. Pendahuluan**


**Sales Performance Analysis**
Sales Performance Analysis dapat membantu e-commerce dalam memahami perilaku customer, mengoptimalkan strategi pemasaran, hingga mengidentifikasi tren yang mendorong pendapatan. Hal ini dilakukan dengan menganalisis beberapa metrik seperti nilai pesanan rata-rata, dan kinerja produk. Hasil dari analisis ini dapat membantu user untuk membuat keputusan berdasarkan data.

**Elasticsearch dan Kibana**
Elasticsearch dan Kibana adalah salah satu tool untuk analisis karena kemampuannya menangani data dalam jumlah besar secara real-time, menyediakan kueri yang cepat, terukur, dan fleksibel. Kemampuan pencarian Elasticsearch yang kuat dipadukan dengan dashboard intuitif dan alat visualisasi Kibana.

Maka dari itu, project ini bertujuan untuk melakukan Sales Performance Analysis menggunakan database Elasticsearch dan dashboard Kibana. Project ini menggunakan data e-commerce dengan sumber di bawah.

Data: https://www.kaggle.com/datasets/vivek468/superstore-dataset-final

## **B. Data Loading**

In [1]:
# Memanggil Module

# Module pengolahan data
import pandas as pd
import numpy as np

# Module Visualisasi
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

# Module Elasticsearch
from elasticsearch import Elasticsearch

In [2]:
# Memanggil data
df_1 = pd.read_csv(r"C:\Users\mnuzu\Downloads\Hacktiv\03_Phase_02\Milestone\Sample_Superstore.csv", delimiter=',', encoding='ISO-8859-1')


## **C. Data Cleaning**

### **1. Mengecek data secara general**

In [3]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

### **2. Melakukan Data Cleaning**

In [5]:
# Fungsi untuk Task 2
def data_cleaning(df):
    # Menghilangkan baris data duplikat
    df = df.drop_duplicates()

    # Mengkecilkan huruf pada header
    df.columns = df.columns.str.lower()

    # Mengubah spasi menjadi simbol garis bawah pada header
    df.columns = df.columns.str.replace(' ', '_')

    # Menghilangkan whitespace pada header
    df.columns = df.columns.str.strip()

    # Menghilangkan simbol kecuali simbol garis bawah pada header
    df.columns = df.columns.str.replace(r'[^\w_]', '', regex=True)

    # Mengatasi missing values
    for column in df.columns:
        if df[column].dtype == 'object':
            # Melakukan imputasi missing values dengan mode jika kolom bertipe kategorikal
            df[column].fillna(df[column].mode()[0], inplace=True)
        else:
            # Melakukan imputasi missing values dengan median jika kolom bertipe numerikal
            df[column].fillna(df[column].median(), inplace=True)

    return df


df_2 = data_cleaning(df_1)
df_2

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


## **D. Importing Data to Elasticsearch Database**

In [8]:
# Memasukkan data ke dalam database elasticsearch dari local

# Inisialisasi koneksi dengan elasticsearch
es = Elasticsearch("http://localhost:9200") 

# Melakukan iterasi per masing-masing baris data
for i,r in df_2.iterrows():
	# Ubah ke dalam json
	doc=r.to_json()
	# Memasukkan data ke elasticsearch dengan index frompostgresql
	res=es.index(index="data_milestone", doc_type="doc", body=doc)
	print(res)

c:\Users\mnuzu\anaconda3\envs\airflow\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'data_milestone', '_type': 'doc', '_id': 'P3I2G5MBrT7R5Ad1bXPh', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
{'_index': 'data_milestone', '_type': 'doc', '_id': 'QHI2G5MBrT7R5Ad1bnNM', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
{'_index': 'data_milestone', '_type': 'doc', '_id': 'QXI2G5MBrT7R5Ad1bnOH', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
{'_index': 'data_milestone', '_type': 'doc', '_id': 'QnI2G5MBrT7R5Ad1bnPD', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
{'_index': 'data_milestone', '_type': 'doc', '_id': 'Q3I2G5MBrT7R5Ad1bnP_', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term'

## **E. Sales Performance Analysis**

Hasil dashboard dan presentasi dapat dilihat di link berikut:
https://drive.google.com/drive/folders/1zoSEYi5OpOShjWoOOCbGm4PMiHaALoc6?usp=sharing

Berikut merupakan hasil dari Sales Performance Analysis yang dilakukan pada Kibana.
* **Most Sold Products**: Produk yang paling banyak terjual adalah perlengkapan kantor seperti stapler, binder, dan kertas.
* **Low-Margin High-Volume Products**: Meskipun binder dan kertas merupakan produk yang paling banyak terjual, produk tersebut tidak memberikan keuntungan signifikan dibandingkan dengan copier dan telepon.
    * **Consequency**: Efisiensi biaya operasional yang rendah.
* **High Loss Products**: Meja adalah produk dengan kerugian yang cukup besar.
    * **Consequency**: Kerugian ini dapat menurunkan margin keuntungan secara keseluruhan. Kerugian ini mungkin disebabkan oleh produk yang rusak saat pengiriman, pembeli membatalkan pesanan, dan kita harus melakukan pengembalian dana.
* **Shipping Service**: Sebagian besar pembeli memilih untuk mengirimkan produk kami menggunakan pengiriman kelas standar.
    * **Consequency**: Hal ini dapat menyebabkan kinerja logistik yang rendah karena banyak orang menggunakan jenis pengiriman yang sama. Pengiriman dapat terlambat, dan produk dapat rusak selama pengiriman. Hal ini mungkin disebabkan oleh tingginya harga pengiriman First Class dan Same-Day.
* **Custumer Location**: Sebagian besar pembeli berasal dari California.
    * **Opportunity**: Kita dapat membangun gudang baru di California untuk mengurangi biaya pengiriman untuk First Class dan Same-Day.




## Recommendations

Berikut merupakan rekomendasi tentang keputusan bisnis yang dapat dijalankan berdasarkan penemuan sebelumnya:
* Untuk menurunkan efisiensi biaya operasional, We need to increase the price for binders and paper.
* Untuk meningkatkan margin keuntungan secara keseluruhanWe need to discontinue the table to be sold on our platform.
* Untuk mengurangi biaya pengiriman untuk First Class dan Same-Day, We need to lower the price for First and Same Day shipment classes by building a new warehouse in California.